In [1]:
%%capture
%pip install langchain_community langchain langchain_qdrant qdrant_client langchain_huggingface

# Import necessary libraries   

In [2]:
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_community.document_loaders import DataFrameLoader
import pandas as pd

import torch
device = "cuda" if torch.cuda.is_available() else  "cpu"

# Custom SQL Store

In [3]:
from pydantic import BaseModel, Field
from typing import Optional

class DocumentModel(BaseModel):
    key: Optional[str] = Field(None)
    page_content: Optional[str] = Field(None)
    metadata: dict = Field(default_factory=dict)

In [4]:
from sqlalchemy import Column, String, create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy.dialects.postgresql import JSONB

Base = declarative_base()

class SQLDocument(Base):
    __tablename__ = "docstore"
    key = Column(String, primary_key=True)
    value = Column(JSONB)

    def __repr__(self):
        return f"<SQLDocument(key='{self.key}', value='{self.value}')>"

# Load data from CSV

In [5]:
import logging
from typing import Generic, Iterator, Sequence, TypeVar
from langchain.schema import Document
from langchain_core.stores import BaseStore

from sqlalchemy.orm import sessionmaker, scoped_session

logger = logging.getLogger(__name__)

D = TypeVar("D", bound=Document)

class PostgresStore(BaseStore[str, DocumentModel], Generic[D]):
    def __init__(self, connection_string: str):
        self.engine = create_engine(connection_string)
        Base.metadata.create_all(self.engine)
        self.Session = scoped_session(sessionmaker(bind=self.engine))

    def serialize_document(self, doc: Document) -> dict:
        return {"page_content": doc.page_content, "metadata": doc.metadata}

    def deserialize_document(self, value: dict) -> Document:
        return Document(page_content=value.get("page_content", ""), metadata=value.get("metadata", {}))


    def mget(self, keys: Sequence[str]) -> list[Document]:
        with self.Session() as session:
            try:
                sql_documents = session.query(SQLDocument).filter(SQLDocument.key.in_(keys)).all()
                return [self.deserialize_document(sql_doc.value) for sql_doc in sql_documents]
            except Exception as e:
                logger.error(f"Error in mget: {e}")
                session.rollback()
                return [] 
    def mset(self, key_value_pairs: Sequence[tuple[str, Document]]) -> None:
        with self.Session() as session:
            try:
                serialized_docs = []
                for key, document in key_value_pairs:
                    serialized_doc = self.serialize_document(document)
                    serialized_docs.append((key, serialized_doc))

                documents_to_update = [SQLDocument(key=key, value=value) for key, value in serialized_docs]
                session.bulk_save_objects(documents_to_update, update_changed_only=True)
                session.commit()
            except Exception as e:
                logger.error(f"Error in mset: {e}")
                session.rollback()


    def mdelete(self, keys: Sequence[str]) -> None:
        with self.Session() as session:
            try:
                session.query(SQLDocument).filter(SQLDocument.key.in_(keys)).delete(synchronize_session=False)
                session.commit()
            except Exception as e:
                logger.error(f"Error in mdelete: {e}")
                session.rollback() 
    def yield_keys(self, *, prefix: Optional[str] = None) -> Iterator[str]:
        with self.Session() as session:
            try:
                query = session.query(SQLDocument.key)
                if prefix:
                    query = query.filter(SQLDocument.key.like(f"{prefix}%"))
                for key in query:
                    yield key[0]
            except Exception as e:
                logger.error(f"Error in yield_keys: {e}")
                session.rollback()

In [6]:
df = pd.read_csv('/kaggle/input/corpus-tvpl-20k/data.csv')
df = df.loc[:, ~df.columns.str.match('Unnamed')]

In [7]:
df = df.drop(['source_id', 'source', 'html_text', 'gazette_number', 'attribute', 'note'], axis=1)

In [8]:
df = df.fillna('None')

In [9]:
first_row = df.iloc[[0]]
first_row

,url,title,full_text,official_number,document_info,document_status,place_issue,signer,document_type,document_field,issued_date,effective_date,enforced_date
0,https://thuvienphapluat.vn/van-ban/Tien-te-Nga...,Thông báo 6905/TB-KBNN,BỘ TÀI CHÍNH \nKHO BẠC NHÀ NƯỚC \n CỘNG HÒA XÃ...,6905/TB-KBNN,Thông báo 6905/TB-KBNN về Tỷ giá hạch toán ngo...,Đã biết,kho bạc nhà nước,Trần Quân,Thông báo,Tiền tệ - Ngân hàng,29/11/2024,Đã biết,Dữ liệu đang cập nhật


In [10]:
rows = df.iloc[:3,:]
rows

,url,title,full_text,official_number,document_info,document_status,place_issue,signer,document_type,document_field,issued_date,effective_date,enforced_date
0,https://thuvienphapluat.vn/van-ban/Tien-te-Nga...,Thông báo 6905/TB-KBNN,BỘ TÀI CHÍNH \nKHO BẠC NHÀ NƯỚC \n CỘNG HÒA XÃ...,6905/TB-KBNN,Thông báo 6905/TB-KBNN về Tỷ giá hạch toán ngo...,Đã biết,kho bạc nhà nước,Trần Quân,Thông báo,Tiền tệ - Ngân hàng,29/11/2024,Đã biết,Dữ liệu đang cập nhật
1,https://thuvienphapluat.vn/cong-van/Bao-hiem/C...,Công văn 17509/BTC-HCSN,BỘ TÀI CHÍNH \n CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT...,17509/BTC-HCSN,Công văn 17509/BTC-HCSN năm 2013 ngân sách nhà...,Đã biết,Bộ Tài chính,Nguyễn Thị Minh,Công văn,"Bảo hiểm, Lao động - Tiền lương, Tài chính nhà...",18/12/2013,Đã biết,Dữ liệu đang cập nhật
2,https://thuvienphapluat.vn/van-ban/Bo-may-hanh...,Kế hoạch 749/KH-UBND,ỦY BAN NHÂN DÂN \nTỈNH QUẢNG BÌNH \n CỘNG HÒA ...,749/KH-UBND,Kế hoạch 749/KH-UBND năm 2022 thực hiện Quyết ...,Đã biết,Tỉnh Quảng Bình,Hồ An Phong,Kế hoạch,Bộ máy hành chính,09/05/2022,Đã biết,Dữ liệu đang cập nhật


In [11]:
loader = DataFrameLoader(rows, page_content_column="full_text")

documents = loader.load()

In [12]:
len(documents[0].page_content)

6458

In [13]:
def flatten(xss):
    return [x for xs in xss for x in xs]

# Load Embeddings Model

In [6]:
embedding = HuggingFaceEmbeddings(
    model_name='hiieu/halong_embedding',
    model_kwargs={'device': device},
    encode_kwargs= {'normalize_embeddings': False}
)

d:\App\Anaconda\envs\psang\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\App\Anaconda\envs\psang\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Chunking Methods

In [7]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=8096*2)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=512)

In [16]:
import uuid
def prepare_parent_chunks(data):
    chunks = []
    base_metadata = {
        "url": data['url'] if data.get('url') else "",
        "title": data['title'] if data.get('title') else "",
        "official_number": data.get("official_number") if data.get("official_number") else "" ,
        "document_info": data.get("document_info") if data.get("document_info") else "",
        "document_status": data.get("document_status") if data.get("document_status") else "",
        "place_issue": data.get("document_info") if data.get("place_issue") else "",
        "signer": data.get("signer") if data.get("signer") else "",
        "document_type": data.get("document_type") if data.get("document_type") else "",
        "document_field": data.get("document_field") if data.get("document_field") else "",
        "issued_date": data.get("issued_date") if data.get("issued_date") else "",
        "effective_date": data.get("effective_date") if data.get("effective_date") else "",
        "enforced_date": data.get("enforced_date") if data.get("enforced_date") else "",
    }

    docs = parent_splitter.split_text(data["full_text"])
    for doc in docs:
        chunks.append({
            "key":str(uuid.uuid4()),
            "page_content": doc,
            "metadata": base_metadata
        })
        
    return chunks

parent_chunks = [prepare_parent_chunks(sample) for sample in df.iloc]
len(parent_chunks)

19367

In [17]:
flatten_parent_chunks = flatten(parent_chunks)
len(flatten_parent_chunks)

37966

In [18]:
def prepare_child_chunks(data):
    chunks = []
    base_metadata = {
        "doc_id": data.get("key"),
        **data.get("metadata")
    }
    text = data.get("page_content")
    docs = child_splitter.split_text(text)
    for doc in docs:
        chunks.append({
            "key":str(uuid.uuid4()),
            "page_content": doc,
            "metadata": base_metadata
        })
        
    return chunks

child_chunks = [prepare_child_chunks(sample) for sample in flatten_parent_chunks]
len(child_chunks)

37966

In [19]:
flatten_child_chunks = flatten(child_chunks)
len(flatten_child_chunks)

271673

# Embedding child chunks

In [20]:
def custom_collate_fn(batch):
    """Collate function to handle variable-sized data."""
    collated_batch = {"metadata": [], "data": []}
    for item in batch:
        collated_batch["metadata"].append(item.get("metadata", {}))
        collated_batch["data"].append(item.get("page_content", None))
    return collated_batch

In [21]:
from torch.utils.data import DataLoader

loader = DataLoader(
    flatten_child_chunks,
    batch_size=32,
    collate_fn=custom_collate_fn,
    shuffle=False,
)

In [22]:
def create_embeddings(batch):
    texts = [chunk for chunk in batch["data"]]
    batch_embeddings = embedding.embed_documents(texts) 
    return batch_embeddings

In [23]:
from concurrent.futures import ThreadPoolExecutor
from torch.utils.data import DataLoader
from tqdm import tqdm

embeddings = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for batch in loader:
        futures.append(executor.submit(create_embeddings, batch))
    
    for future in tqdm(futures, desc="Embedding batches"):
        embeddings.extend(future.result())

Embedding batches: 100%|██████████| 8490/8490 [1:17:20<00:00,  1.83it/s]


In [35]:
len(embeddings)

271673

In [36]:
for chunk, embedding in zip(flatten_child_chunks, embeddings):
    chunk["embedding"] = embedding

In [37]:
df_embeded = pd.DataFrame(data=flatten_child_chunks)
df_embeded.head()

,key,page_content,metadata,embedding
0,602a02b4-37fe-498d-936e-c489ba1bc138,BỘ TÀI CHÍNH \nKHO BẠC NHÀ NƯỚC \n CỘNG HÒA XÃ...,{'doc_id': 'e0804ac4-6ee1-432b-b9fa-fbba4f7ae2...,"[0.0407029390335083, 0.07425945997238159, -0.0..."
1,d98d9f5c-3501-4496-8268-ef542983776b,"Nơi nhận: \n VPQH, VPCP, VP CTN; \n Viện KSNDT...",{'doc_id': 'e0804ac4-6ee1-432b-b9fa-fbba4f7ae2...,"[0.063381627202034, 0.0036498084664344788, -0...."
2,2d795588-55b8-4b45-aa2b-b1bf6b9b12bd,1 UAE DIRHAM AED 6.609 \n2 AFGHAN AFGHANI AFN ...,{'doc_id': 'e0804ac4-6ee1-432b-b9fa-fbba4f7ae2...,"[0.023174073547124863, 0.02648521400988102, -0..."
3,968e04d8-1422-4cb8-bccd-71d33995f9f7,59 GUINEA FRANC GNF 3 \n60 QUETZAL GTQ 3.149 \...,{'doc_id': 'e0804ac4-6ee1-432b-b9fa-fbba4f7ae2...,"[0.02090383879840374, 0.02827535569667816, -0...."
4,77e055c6-c7cf-45f1-8f50-c7bd20a2a385,121 ZLOTY PLN 5.960 \n122 GUARANI PYG 3 \n123 ...,{'doc_id': 'e0804ac4-6ee1-432b-b9fa-fbba4f7ae2...,"[0.020989082753658295, 0.05037805810570717, 0...."


In [8]:
def convert_numpy_types(obj):
    """Convert numpy types to Python native types"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    return obj

# Connect to Qdrant & PostgresSQL

In [10]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()

# QDRANT_API = user_secrets.get_secret("Qdrant_Demo_GraphRAG")
QDRANT_API = "SjRcv7rzROCd-HD59KQRfy7mHgWrLyn0LBaWfQ5zIBNLDns4OlFjHg"
QDRANT_URL = "https://21a75178-7457-4e63-974b-666f8174af84.us-west-2-0.aws.cloud.qdrant.io:6333"
QDRANT_COLLECTION = 'corpus_tvpl'

In [11]:
client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API
)

In [42]:
from qdrant_client.models import VectorParams, Distance, CollectionConfig

client.create_collection(
   collection_name="corpus_tvpl",
    vectors_config=VectorParams(
       size=768,  # Kích thước vector của embedding (chỉnh sửa theo embedding của bạn)
       distance=Distance.COSINE,  # Khoảng cách cosine là phổ biến cho các embedding dense
   ),
)
print("Collection đã được tạo thành công.")

Collection đã được tạo thành công.


In [14]:
from dotenv import load_dotenv
import os

load_dotenv()


# URL Ngrok
NGROK_URL = "0.tcp.ap.ngrok.io"
NGROK_PORT = 11865
DB_URL = "localhost:5432"

# Thông tin PostgreSQL
DB_NAME = 'parent'
DB_USER = 'postgres'
# DB_PASSWORD = user_secrets.get_secret("pw_db_postgre")
DB_PASSWORD = os.getenv("DB_PASSWORD") 

In [15]:
URL_STRING = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}/{DB_NAME}"
document_store = PostgresStore(URL_STRING)

In [16]:
vector_store = QdrantVectorStore(
    client=client, 
    collection_name='corpus_tvpl',
    embedding=embedding
)

# Add Documents to Qdrant

In [46]:
from qdrant_client.models import PointStruct
import numpy as np 

batch_size = 1000

points = [
    PointStruct(
        id=row["key"],  # Convert numpy.int64 to string
        vector=convert_numpy_types(row["embedding"]),  # Convert numpy array to list if needed
        payload={
            "page_content": row["page_content"],
            "metadata": convert_numpy_types(row["metadata"])
        }
    )
    for _, row in df_embeded.iterrows()
]

In [47]:
for i in range(0, len(points), batch_size):
    batch_points = points[i:i + batch_size]
    try:
        vector_store.client.upsert(
            collection_name="corpus_tvpl",
            points=batch_points
        )
        if (i//batch_size + 1) % 50 == 0:
            print(f"Batch {i//batch_size + 1} uploaded successfully ({len(batch_points)} points)")
    except Exception as e:
        print(f"Error uploading batch {i//batch_size + 1}: {str(e)}")

print("Data upload completed.")

Batch 50 uploaded successfully (1000 points)
Batch 100 uploaded successfully (1000 points)
Batch 150 uploaded successfully (1000 points)
Batch 200 uploaded successfully (1000 points)
Batch 250 uploaded successfully (1000 points)
Data upload completed.


# Add Documents to PostgresSQL

In [48]:
df_flatten_parent_chunks = pd.DataFrame(flatten_parent_chunks)
print(df_flatten_parent_chunks.shape)
df_flatten_parent_chunks.head(2)

(37966, 3)


,key,page_content,metadata
0,e0804ac4-6ee1-432b-b9fa-fbba4f7ae20a,BỘ TÀI CHÍNH \nKHO BẠC NHÀ NƯỚC \n CỘNG HÒA XÃ...,{'url': 'https://thuvienphapluat.vn/van-ban/Ti...
1,9a30a9e5-fdb6-407d-b674-f0185a25741c,BỘ TÀI CHÍNH \n CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT...,{'url': 'https://thuvienphapluat.vn/cong-van/B...


In [49]:
import pandas as pd
from sqlalchemy import create_engine
import io

engine = create_engine(URL_STRING)

In [50]:
df_flatten_parent_chunks.iloc[0]['metadata']

{'url': 'https://thuvienphapluat.vn/van-ban/Tien-te-Ngan-hang/Thong-bao-6905-TB-KBNN-2024-Ty-gia-hach-toan-ngoai-te-thang-12-635137.aspx',
 'title': 'Thông báo 6905/TB-KBNN',
 'official_number': '6905/TB-KBNN',
 'document_info': 'Thông báo 6905/TB-KBNN về Tỷ giá hạch toán ngoại tệ tháng 12 năm 2024 do Kho bạc Nhà nước ban hành',
 'document_status': 'Đã biết',
 'place_issue': 'Thông báo 6905/TB-KBNN về Tỷ giá hạch toán ngoại tệ tháng 12 năm 2024 do Kho bạc Nhà nước ban hành',
 'signer': 'Trần Quân',
 'document_type': 'Thông báo',
 'document_field': 'Tiền tệ - Ngân hàng',
 'issued_date': '29/11/2024',
 'effective_date': 'Đã biết',
 'enforced_date': 'Dữ liệu đang cập nhật'}

In [51]:
import json

df_processed = [
    {
        "key": row.get('key'),
        "value": {
            **row.get('metadata'),
            'page_content': row.get('page_content')
        }
    }
    for _, row in df_flatten_parent_chunks.iterrows()
]
df_processed = pd.DataFrame(df_processed)
df_processed['value'] = df_processed['value'].apply(json.dumps)

In [52]:
try:
    df_processed.to_sql(
        'docstore',
        engine,
        if_exists='append',
        index=False,
        method='multi',  
        chunksize=1000   
    )
except Exception as e:
    print(f"Error: {e}")
    engine.rollback()  # Rollback the transaction
finally:
    engine.dispose()  # Ensure connection is closed


# Retriever

In [58]:
document_store = PostgresStore(URL_STRING)

In [59]:
len(list(document_store.yield_keys()))

37966

In [17]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=document_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

In [18]:
query = "Chạy vượt đèn đỏ"
query_vector = embedding.embed_query(query)
print("Query vector:", query_vector[:5])


Query vector: [0.05239662528038025, -0.05257686600089073, -0.05722544342279434, -0.03124828264117241, 0.010826312936842442]


In [19]:
results = vector_store.similarity_search(query, k=5)
results

[Document(metadata={'doc_id': '8151cbcf-438d-48fc-8073-e02a051205d6', 'url': 'https://thuvienphapluat.vn/van-ban/Giao-thong-Van-tai/Thong-tu-05-2024-TT-BGTVT-sua-doi-Thong-tu-van-tai-duong-bo-dich-vu-ho-tro-van-tai-duong-bo-604598.aspx', 'title': 'Thông tư 05/2024/TT-BGTVT', 'official_number': '05/2024/TT-BGTVT', 'document_info': 'Thông tư 05/2024/TT-BGTVT sửa đổi Thông tư liên quan đến lĩnh vực vận tải đường bộ, dịch vụ hỗ trợ vận tải đường bộ, phương tiện và người lái do Bộ trưởng Bộ Giao thông vận tải ban hành', 'document_status': 'Đã biết', 'place_issue': 'Thông tư 05/2024/TT-BGTVT sửa đổi Thông tư liên quan đến lĩnh vực vận tải đường bộ, dịch vụ hỗ trợ vận tải đường bộ, phương tiện và người lái do Bộ trưởng Bộ Giao thông vận tải ban hành', 'signer': 'Nguyễn Duy Lâm', 'document_type': 'Thông tư', 'document_field': 'Giao thông - Vận tải', 'issued_date': '31/03/2024', 'effective_date': 'Đã biết', 'enforced_date': '19/04/2024', '_id': '9ffbd705-9446-4c12-b529-19eb3ae15f26', '_collecti

In [22]:
parent_document_retriever.invoke("Chạy xe vượt đèn đỏ bị xử phạt như thế nào")

[Document(metadata={'url': 'https://thuvienphapluat.vn/van-ban/Vi-pham-hanh-chinh/Nghi-dinh-100-2019-ND-CP-xu-phat-vi-pham-hanh-chinh-linh-vuc-giao-thong-duong-bo-va-duong-sat-426369.aspx', 'title': 'Nghị định 100/2019/NĐ-CP', 'signer': 'Nguyễn Xuân Phúc', 'issued_date': '30/12/2019', 'place_issue': 'Nghị định 100/2019/NĐ-CP quy định về xử phạt vi phạm hành chính trong lĩnh vực giao thông đường bộ và đường sắt'}, page_content='e) Không gắn biển báo hiệu ở phía trước xe kéo, phía sau xe được kéo; điều\nkhiển xe kéo rơ moóc không có biển báo hiệu theo quy định;\n\ng) Bấm còi trong đô thị và khu đông dân cư trong thời gian từ 22 giờ ngày hôm\ntrước đến 05 giờ ngày hôm sau, trừ các xe ưu tiên đang đi làm nhiệm vụ theo\nquy định.\n\n2. Phạt tiền từ 400.000 đồng đến 600.000 đồng đối với người điều khiển xe\nthực hiện một trong các hành vi vi phạm sau đây:\n\na) Chuyển làn đường không đúng nơi cho phép hoặc không có tín hiệu báo trước,\ntrừ các hành vi vi phạm quy định tại điểm g khoản 5 Điều